# Redis Product Catalog

## Product Catalog Use Case

In the last session several common Redis use cases were discussed. Now, lets consider using Redis as our primary database for a product catalog for a new online store. It will need to store product details including a name, description, vendor, price, main category and some images.

## Requirements:

* Product information stored in the database should include: name, description, vendor, price, category, images associated with that product.
* Ability to create/update/delete product details
* Ability to find product by ID
* Ability to find products in category X
* Ability to find product by its name or part of its name

## Step 3 - Logical Data Model:

The logical data model is separate from the DBMS being used.
It defines the structure of data elements and to set relationships between them.

* Product Image
    * Id: Number
    * Value: Binary

* Product
    * Id: Number
    * Name: String
    * Description: String
    * Vendor: String
    * Price: Number
    * Currency: String
    * MainCategory: Category (1)
    * Images: Image (0..n)
 
* Category
    * id: Number
    * Name: String
    * Products: Product (0..n)

In [1]:
# Lets make some sample products in a json:

## Step 4 - Convert to Physical Data Model for Redis:

How would you take this logical data model and turn it into a physical model in Redis?

What data structures would you use to meet the requirements listed above?

How to index and query the data?

Be ready to show via redis-cli (or Insight) how you stored the data and how you are able to find the product by ID, list all prodcuts in a given category and find a Product by full or partial name.

In [1]:
# import redis
import redis
import os

# connect redis to localhost and port
redis = redis.Redis(host='localhost', port=6379, db=0)

In [2]:
# it looks like we can store it as a simple String (key value pair)
# we will need to fix the structure how how we deal with this though.

In [3]:
redis.flushdb()

True

# Product Catalog Testing

In [1]:
import redis
import os
import abc, json, pathlib
from abc import abstractmethod
from dataclasses import dataclass, field
import numpy as np
from pathlib import Path
# for type annotations
from typing import Any, Dict, List, Optional, Union, Collection
PathOrStr = Union[Path,str]

In [2]:
# Utils

def if_none(a: Any, b: Any) -> Any:
    "`a` if `a` is not None, otherwise `b`."
    return b if a is None else a


def to_type(a: Any, data_type):
    """
    If item is None, return None, else, convert to an data_type specified
    (ie. np.array, str, int, float, ect..)
    :parameter: a (Any or None)
    :returns: None or data_type(a)
    """
    return None if a is None else data_type(a)

In [3]:
def redis(host='localhost', port=6379, db=0):
    return redis.Redis(host=host, port=port, db=db)

In [4]:

redis = redis_db(host='localhost', port=6379, db=0)
redis

Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>

In [5]:

class DataObject(metaclass=abc.ABCMeta):
    """
    A abstract base class to work with subclasses `DeviationSurvey` and `CalculableObject`.
    """
    @abstractmethod
    def from_json(self):
        pass

    @abstractmethod
    def validate(self):
        pass

    @abstractmethod
    def deserialize(self):
        pass

    @abstractmethod
    def serialize(self):
        pass

In [6]:
@dataclass
class ProductCreate(DataObject):
    """
    Dataclass for Product
    :parameter:
    productId:    (required) product id
    name:         (required) product name
    description:  (required) product description
    vendor:       (required) product vendor
    price:        (required) product price
    currency:     (required) currency
    category:     (required) product category
    images:       (required) images associated with product (binary value)
    :returns:
    dataclassObj: Dataclass Product object
    """
    #productId: str
    name: str
    description: str
    vendor: str
    price: float
    currency: str
    category: str
    productId: str = field(default=None, metadata={'unit': 'str'})
    images: np.ndarray = field(default=None, metadata={'unit': 'str'})

    def from_json(self):
        super().from_json()

    def serialize(self):
        super().serialize()
    
    def validate(self):
        """
        validate different parameters to ensure that the data in the DataObject
        will work with the directional survey functions
        """


#         def validate_productId(self):
#             """
#             validate that productId is a string
#             :return: pass or TypeError
#             """
#             productId_type = type(self.productId)
#             if productId_type is str:
#                 pass
#             else:
#                 raise TypeError(f"Validation Error: productId has type {productId_type}")
        
        def validate_product_name(self):
            """
            validate that the product name has not been used
            :return: pass or TypeError
            """
            product_name = self.name
            print(product_name)
            if redis.sadd('product-names',product_name) == 1:
                pass
            else:
                raise TypeError(f"Validation Error: product name must be unique: {product_name}")
                
        # run validation functions
#         validate_productId(self)
        validate_product_name(self)

    def deserialize(self):
        """
        convert dict values to their proper deserialized dict values
        converts lists to np.arrays if not None
        converts value to float if not None
        converts value to str if not None
        :parameter:
        DataObject params
        :return:
        DataObject params deserialized as floats, str, int, or np.arrays
        """

        self.productId = to_type(self.productId, str)
        self.name = to_type(self.name, str)
        self.description = to_type(self.description, str)
        self.vendor = to_type(self.vendor, str)
        self.price = to_type(self.price, float)
        self.currency = to_type(self.currency, str)
        self.category = to_type(self.category, str)
        self.images = to_type(self.images, np.array)

    def __post_init__(self):
        """
        validate all data,
        serialized all validated data,
        look in all fields and types,
        if type is None pass,
        else if type given doesnt match dataclass type raise error
        """
        self.validate()
        self.deserialize()
        print(self.deserialize())
        for (name, field_type) in self.__annotations__.items():
            if not isinstance(self.__dict__[name], field_type):
                current_type = type(self.__dict__[name])
                if current_type is type(None):
                    pass
                else:
                    raise ValueError(f"The field `{name}` was assigned by `{current_type}` instead of `{field_type}`")


In [7]:
@dataclass
class ProductUpdate(DataObject):
    """
    Dataclass for Product
    :parameter:
    productId:    (required) product id
    name:         (required) product name
    description:  (required) product description
    vendor:       (required) product vendor
    price:        (required) product price
    currency:     (required) currency
    category:     (required) product category
    images:       (required) images associated with product (binary value)
    :returns:
    dataclassObj: Dataclass Product object
    """

    productId: str
    name: str = field(default=None, metadata={'unit': 'str'})
    description: str = field(default=None, metadata={'unit': 'str'})
    vendor: str = field(default=None, metadata={'unit': 'str'})
    price: float = field(default=None, metadata={'unit': 'float'})
    currency: str = field(default=None, metadata={'unit': 'str'})
    category: str = field(default=None, metadata={'unit': 'str'})
    images: np.ndarray = field(default=None, metadata={'unit': 'str'})

    def from_json(self):
        super().from_json()

    def serialize(self):
        super().serialize()

    def validate(self):
        """
        validate different parameters to ensure that the data in the DataObject
        will work with the directional survey functions
        """


        def validate_productId(self):
            """
            validate that productId is a string
            :return: pass or TypeError
            """
            print(self.productId)
            print(redis.hget(self.productId,'name'))
            if type(redis.hget(self.productId,'name')) is bytes:
                pass
            else:
                raise TypeError(f"Validation Error: productId does not exist")
        
#         def validate_product_name(self):
#             """
#             validate that the product name has not been used
#             :return: pass or TypeError
#             """
#             product_name = self.name
#             print(product_name)
#             if redis.sadd('product-names',product_name) == 1:
#                 pass
#             else:
#                 raise TypeError(f"Validation Error: product name must be unique: {product_name}")
                
        # run validation functions
        validate_productId(self)
        #validate_product_name(self)

    def deserialize(self):
        """
        convert dict values to their proper deserialized dict values
        converts lists to np.arrays if not None
        converts value to float if not None
        converts value to str if not None
        :parameter:
        DataObject params
        :return:
        DataObject params deserialized as floats, str, int, or np.arrays
        """

        self.productId = to_type(self.productId, str)
        self.name = to_type(self.name, str)
        self.description = to_type(self.description, str)
        self.vendor = to_type(self.vendor, str)
        self.price = to_type(self.price, float)
        self.currency = to_type(self.currency, str)
        self.category = to_type(self.category, str)
        self.images = to_type(self.images, np.array)

    def __post_init__(self):
        """
        validate all data,
        serialized all validated data,
        look in all fields and types,
        if type is None pass,
        else if type given doesnt match dataclass type raise error
        """
        self.validate()
        self.deserialize()
        print(self.deserialize())
        for (name, field_type) in self.__annotations__.items():
            if not isinstance(self.__dict__[name], field_type):
                current_type = type(self.__dict__[name])
                if current_type is type(None):
                    pass
                else:
                    raise ValueError(f"The field `{name}` was assigned by `{current_type}` instead of `{field_type}`")


In [8]:
class CalculableObject(DataObject):

    def __init__(self, product_obj, **kwargs):
        """
        DirectionalSurvey object with a wells directional survey info
        Attributes:
        directional_survey_points (Dataclass Object) DataObject object
        """

        self.product_obj = product_obj

    def validate(self):
        super().validate()

    def deserialize(self):
        super().deserialize()

    @classmethod
    def from_json(cls, path: PathOrStr):
        """
        Pass in a json path, either a string or a Path lib path and convert to a WellboreTrajectory data obj
        :param:
        -------
         path: PathOrStr
        :return:
        -------
        deviation_survey_obj: Obj
        :examples:
        -------
        """

        with open(path) as json_file:
            json_data = json.load(json_file)
        json_file.close()

        res = cls(data=json_data)  # converts json data
        return res

    def serialize(self):
        """
        Convert survey object to serialized json
        :parameter:
        -------
        None
        :return:
        -------
        json: str
        :examples:
        -------
        """
        
        self.productId = to_type(self.productId, str)
        self.name = to_type(self.name, str)
        self.description = to_type(self.description, str)
        self.vendor = to_type(self.vendor, str)
        self.price = to_type(self.price, float)
        self.currency = to_type(self.currency, str)
        self.category = to_type(self.category, str)
        self.images = to_type(self.images, np.array)

        json_obj = dict(productId=str(self.product_obj.productId),
                        name=str(self.product_obj.name),
                        description=str(self.product_obj.description),
                        vendor=str(self.product_obj.vendor),
                        price=float(self.product_obj.price),
                        currency=str(self.product_obj.currency),
                        images=list(self.product_obj.images))


        json_string = json.dumps(json_obj)  # converts a data object into a json string.

        return json_string

In [9]:
class ProductCatalogCreate(CalculableObject):

    def __init__(self, data=None):
        """
        DirectionalSurvey object with a wells directional survey info
        Attributes:
        directional_survey_points (Dataclass Object) DataObject object
        """

        self.data = data
        self.product_obj = ProductCreate(**self.data)
        
#     def category_mapping(self):
#         "create a mapping for the category"
        
        
    def product_id_gen(self):
        "Generate the product Id"
        product_id_incr = redis.incr('product_id_incr', 1)
        product_id = f"product:{product_id_incr}"
        
        self.product_obj.productId = product_id

    def product_hash(self):
        """
        Calculate TVD, n_s_deviation, e_w_deviation, and dls values along the wellbore
        using md, inc, and azim arrays
        :parameter:
        -------
        None
        :return:
        -------
        calculated np.array values
        tvd: np.array
        dls: np.array
        e_w_deviation: np.array
        n_s_deviation: np.array
        :examples:
        -------
        """
        # get md, inc, and azim arrays
        product_id = self.product_obj.productId
        name = self.product_obj.name
        description = self.product_obj.description
        vendor = self.product_obj.vendor
        price = self.product_obj.price
        currency = self.product_obj.currency
        category = self.product_obj.category
        images = self.product_obj.images
        
        redis.hset(product_id,'name',name)
        redis.hset(product_id,'description',description)
        redis.hset(product_id,'vendor',vendor)
        redis.hset(product_id,'price',price)
        redis.hset(product_id,'currency',currency)
        redis.hset(product_id,'main-category',category)
        
        # create a list of images, keep only 4 per product
        # first image in the list is the number 1 image
        list_id_images = f"{product_id}:images"
        for image_binary_val in images:
            redis.rpush(list_id_images,str(image_binary_val))
            redis.ltrim(list_id_images,0,3)
            #redis.lrange(image_list_id,0,-1)
            
    def category_set(self):
        """
        You can get the ID from the product hash and create the cateogry hash
        you will call the current main cat product id. Find out what it is, then create a unique ID for it.
        Like, MEAT, and then update the set??? idk. ughh
        """
        product_id = self.product_obj.productId
        category = self.product_obj.category
        
        category_id = f"category:{category}"
        redis.sadd(category_id,product_id)
        

    def generate_product_catalog(self):

        self.product_id_gen()  # get generated product id

        self.product_hash()  # get product hash
        
        self.category_set() # get category set

In [10]:
class ProductCatalogUpdate(CalculableObject):

    def __init__(self, data=None):
        """
        DirectionalSurvey object with a wells directional survey info
        Attributes:
        directional_survey_points (Dataclass Object) DataObject object
        """

        self.data = data
        self.product_obj = ProductUpdate(**self.data)
        

    def product_hash_update(self):
        """
        Calculate TVD, n_s_deviation, e_w_deviation, and dls values along the wellbore
        using md, inc, and azim arrays
        :parameter:
        -------
        None
        :return:
        -------
        calculated np.array values
        tvd: np.array
        dls: np.array
        e_w_deviation: np.array
        n_s_deviation: np.array
        :examples:
        -------
        """
        # get md, inc, and azim arrays
        product_id = self.product_obj.productId
        name = self.product_obj.name
        description = self.product_obj.description
        vendor = self.product_obj.vendor
        price = self.product_obj.price
        currency = self.product_obj.currency
        category = self.product_obj.category
        images = self.product_obj.images
        
        # update name if not none, this involves removing and replacing a product name set value
        if name is not None: 
            # get the original value of the name in the product id
            orig_name = redis.hget(product_id,'name')
            # remove it from the set product-names, (must decode bytes to string)
            orig_name = orig_name.decode("utf-8")
            redis.srem('product-names', orig_name)
            # add it the new name to the product-names set
            redis.sadd('product-names',name)
            
            # update the name in the product hash
            redis.hset(product_id,'name',name)
            
        # updates if present    
        if description is not None: redis.hset(product_id,'description',description)
        if vendor is not None: redis.hset(product_id,'vendor',vendor)
        if price is not None: redis.hset(product_id,'price',price)
        if currency is not None: redis.hset(product_id,'currency',currency)
        
        # image updates
        if images is not None:
            list_id_images = f"{product_id}:images"
            # reverse the nparray to keep the image order, (1st in list is best image)
            for image_binary_val in images[::-1]:
                # lpush image items in
                redis.lpush(list_id_images,str(image_binary_val))
                # ltrim the list after the 4th image
                redis.ltrim(list_id_images,0,3)
                #redis.lrange(image_list_id,0,-1)
        
    def category_update(self):
        """
        You can get the ID from the product hash and create the cateogry hash
        you will call the current main cat product id. Find out what it is, then create a unique ID for it.
        Like, MEAT, and then update the set??? idk. ughh
        """
        product_id = self.product_obj.productId
        category = self.product_obj.category
        
        if category is not None:
            
            # get the original value of the category
            orig_cat = redis.hget(product_id,'main-category')
            # remove it from the set, (must decode bytes to string)
            orig_cat = orig_cat.decode("utf-8")
            orig_cat_id = f"category:{orig_cat}"
            redis.srem(orig_cat_id, product_id)
            
            # update in the product hash
            redis.hset(product_id,'main-category',category)
            
            # add it to the new set
            category_id = f"category:{category}"
            redis.sadd(category_id,product_id)

            
            
    def product_hash_delete(self):
        """
        Calculate TVD, n_s_deviation, e_w_deviation, and dls values along the wellbore
        using md, inc, and azim arrays
        :parameter:
        -------
        None
        :return:
        -------
        calculated np.array values
        tvd: np.array
        dls: np.array
        e_w_deviation: np.array
        n_s_deviation: np.array
        :examples:
        -------
        """
        # get md, inc, and azim arrays
        product_id = self.product_obj.productId
        

        # get the original value of the name in the product id
        orig_name = redis.hget(product_id,'name')
        # remove it from the set product-names, (must decode bytes to string)
        orig_name = orig_name.decode("utf-8")
        redis.srem('product-names', orig_name)
            
        # delete the hash
        redis.delete(product_id)
        
    def images_list_delete(self):
        
        product_id = self.product_obj.productId
        #images = self.product_obj.images
        

        list_id_images = f"{product_id}:images"
        redis.delete(list_id_images)
                

    def category_remove(self):
        """
        You can get the ID from the product hash and create the cateogry hash
        you will call the current main cat product id. Find out what it is, then create a unique ID for it.
        Like, MEAT, and then update the set??? idk. ughh
        """
        product_id = self.product_obj.productId

        # get the original value of the category
        orig_cat = redis.hget(product_id,'main-category')
        # remove it from the set, (must decode bytes to string)
        orig_cat = orig_cat.decode("utf-8")
        orig_cat_id = f"category:{orig_cat}"
        redis.srem(orig_cat_id, product_id)           

    def update_product_catalog(self):

        self.product_hash_update()  # get product hash
        
        self.category_update() # get category set
        
    def delete_product_catalog(self):
        
        self.category_remove()
        self.product_hash_delete()
        self.images_list_delete()

In [11]:
# testing

In [12]:


# example dict
product_dict = {
    "name": 'Product_b',
    "description": 'Chicken',
    "vendor": 'WholeFoods',
    "price": 15.5,
    "currency": 'dollars',
    "category": 'Meat',
    "images": [1,2,3,4,5,6,7,8]
}


product_obj = ProductCatalogCreate(product_dict) # get product object
print('product.data:')
print(product_obj.data)
#product_obj.calculate_survey_points() # runs through min curve algo, calc lat lon points, and calc horizontal
#print('calc object:')

product_obj.generate_product_catalog()

Product_b
None
product.data:
{'name': 'Product_b', 'description': 'Chicken', 'vendor': 'WholeFoods', 'price': 15.5, 'currency': 'dollars', 'category': 'Meat', 'images': [1, 2, 3, 4, 5, 6, 7, 8]}


In [13]:
redis.hgetall('product:1')

{b'name': b'Product_b',
 b'description': b'Chicken',
 b'vendor': b'WholeFoods',
 b'price': b'15.5',
 b'currency': b'dollars',
 b'main-category': b'Meat'}

In [14]:
redis.lrange('product:1:images',0,-1)

[]

In [15]:
redis.smembers('category:Meat')

{b'product:2'}

In [16]:
redis.smembers('product-names')

{b'Product_A', b'Product_b'}

In [18]:
# example dict
product_dict = {
    "productId": 'product:1',
    "name": 'product_SICK',
    "description": 'ChickenBUTTS',
    "vendor": 'WholeFoodsBUTSS',
    "price": 1202,
    "currency": 'dollarsZZZZ',
    "category": 'Fish',
    "images": [233,2450]
}

product_obj = ProductCatalogUpdate(product_dict) # get product object
product_obj.update_product_catalog()

product:2
b'Product_b'
None


In [19]:

print(redis.hgetall('product:1'))
print(redis.lrange('product:1:images',0,-1))
print(redis.smembers('category:Meat'))
print(redis.smembers('product-names'))
print(redis.smembers('category:Fish'))

{}
[]
set()
{b'Product_A', b'product_SICK'}
{b'product:2'}


In [34]:
# example dict
product_dict = {
    "productId": 'product:1'
}

product_obj = ProductCatalogUpdate(product_dict) # get product object
product_obj.delete_product_catalog()

product:1
b'product_SICK'
None


In [35]:
print(redis.hgetall('product:1'))
print(redis.lrange('product:1:images',0,-1))
print(redis.smembers('category:Meat'))
print(redis.smembers('product-names'))

{}
[]
set()
set()


In [20]:
redis.flushdb()


True

In [425]:
redis.scan(0)
#redis.smembers('product-names')

(1,
 [b'3748912688',
  b'product:4:images',
  b'product:1:images',
  b'product:2:images',
  b'category:Fish',
  b'product:2',
  b'product:3',
  b'product:5',
  b'product-names',
  b'product:3:images'])